In [1]:
import pandas as pd
import numpy as np
import jsonlines
import seaborn as sns
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
def read_json_lines_into_df(file):
    lines = []
    with jsonlines.open(file) as reader:
        for obj in reader:
            lines.append(obj)
    return pd.DataFrame.from_records(lines)

In [3]:
dev = read_json_lines_into_df('../data/dev.jsonl')
train = read_json_lines_into_df('../data/train.jsonl')
test = read_json_lines_into_df('../data/test.jsonl')

In [4]:
dev.shape
train.shape
test.shape

(500, 4)

(8500, 4)

(1000, 3)

In [5]:
train.label.value_counts()
dev.label.value_counts()

0    5450
1    3050
Name: label, dtype: int64

1    250
0    250
Name: label, dtype: int64

In [6]:
columns = train.columns
print(columns)

Index(['id', 'img', 'label', 'text'], dtype='object')


In [7]:
train.shape

(8500, 4)

In [8]:
from transformers import AlbertModel, AlbertTokenizer
import torch
tokenizer = AlbertTokenizer.from_pretrained('albert-base-v2')
tokenizer.encode("Hello, my dog is cute", add_special_tokens=True)



[2, 10975, 15, 51, 1952, 25, 10901, 3]

In [19]:
tokenised_lengths = train.text.apply(lambda x: tokenizer.encode(x, add_special_tokens=True)).apply(len)

In [23]:
tokenised_lengths.describe()
tokens = torch.tensor(np.vstack(train.text.apply(lambda x: np.array(tokenizer.encode(x, add_special_tokens=True, pad_to_max_length=True, max_length=128))).values))
tokens.shape

count    8500.000000
mean       16.795059
std         9.058886
min         3.000000
25%        10.000000
50%        15.000000
75%        21.000000
max        87.000000
Name: text, dtype: float64

torch.Size([8500, 128])

In [13]:
model = AlbertModel.from_pretrained('albert-base-v2')

In [ ]:
outputs = model(tokens[:128])
outputs[0].shape

In [1]:
print("Hi")



Hi


NameError: name 'outputs' is not defined

In [3]:
from transformers import AlbertModel, AlbertTokenizer
import torch

tokenizer = AlbertTokenizer.from_pretrained('albert-base-v2')
model = AlbertModel.from_pretrained('albert-base-v2')
input_ids = torch.tensor(tokenizer.encode("Hello, my dog is cute", add_special_tokens=True)).unsqueeze(0)  # Batch size 1
outputs = model(input_ids)
last_hidden_states = outputs[0]

In [11]:
input_ids.shape
last_hidden_states.shape
len(outputs)
for o in outputs:
    print(o.shape, type(o))

torch.Size([1, 8])

torch.Size([1, 8, 768])

2

torch.Size([1, 8, 768]) <class 'torch.Tensor'>
torch.Size([1, 768]) <class 'torch.Tensor'>
